In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
import psycopg2
import os
conn = psycopg2.connect(os.environ['POSTGRE_PASSWORD'])
conn.commit()
conn.close()

In [13]:
#使用with as語法，可以自動commit()，自動close()
#使用with，僅能自動commit()，最後需手動輸入close()
#1個分號 = 1個命令
with psycopg2.connect(os.environ['POSTGRE_PASSWORD']) as conn:
    cursor = conn.cursor()
    sql = '''
        CREATE TABLE IF NOT EXISTS 客戶(
        客戶_id SERIAL,
        客戶名稱 VARCHAR(255) NOT NULL,
        PRIMARY KEY(客戶_id)
        );
    '''
    cursor.execute(sql)

    sql = '''
        CREATE TABLE IF NOT EXISTS 聯絡人(
        聯絡人id SERIAL,
        客戶id INT,
        聯絡人姓名 VARCHAR(255) NOT NULL,
        電話 VARCHAR(15),
        郵件 VARCHAR(100),
        PRIMARY KEY(聯絡人id),
        FOREIGN KEY(客戶id) REFERENCES 客戶(客戶_id)
        );
    '''
    cursor.execute(sql)
    cursor.close()

In [12]:
#插入資訊至Table –方法1：手動commit()、手動close()
#語言如上

#插入客戶資訊至Table –方法1：自動commit()、手動close()
conn = psycopg2.connect(os.environ['POSTGRE_PASSWORD'])
with conn:
    with conn.cursor() as cursor:
        sql='''
            INSERT INTO 客戶(客戶名稱)
            VALUES('遠傳電信'),
            ('台灣大車隊');
        '''
        cursor.execute(sql)
    
conn.close()

In [11]:
#插入聯絡人資訊至Table -方法2：全部自動commit()、自動close()
with psycopg2.connect(os.environ['POSTGRE_PASSWORD']) as conn:
    with conn.cursor() as cursor:
        sql='''
            INSERT INTO 聯絡人(客戶id,聯絡人姓名,電話,郵件)
            VALUES (1,'John Doe','(408)-111-1234','john.doe@bluebird.dev'),
	        (1,'Jane Doe','(408)-111-1235','jane.doe@bluebird.dev'),
	        (2,'David Wright','(408)-222-1234','david.wright@dolphin.dev');
        '''
        cursor.execute(sql)

In [10]:
#透過Python
#-建立關聯式(Left Join)Table
#-取出SQL資料庫的值

with psycopg2.connect(os.environ['POSTGRE_PASSWORD']) as conn:
    with conn.cursor() as cursor:
        sql='''
            SELECT 聯絡人id,客戶名稱,聯絡人姓名,電話,郵件
            FROM 聯絡人 LEFT JOIN 客戶 ON 聯絡人.客戶id = 客戶.客戶_id
        '''
        cursor.execute(sql)
        datas:list = cursor.fetchmany(10)

for item in datas:
    print(item)

(1, '遠傳電信', 'John Doe', '(408)-111-1234', 'john.doe@bluebird.dev')
(2, '遠傳電信', 'Jane Doe', '(408)-111-1235', 'jane.doe@bluebird.dev')
(3, '台灣大車隊', 'David Wright', '(408)-222-1234', 'david.wright@dolphin.dev')


In [9]:
# 補充：不知道取出的值有幾筆時，取出值的語法
with psycopg2.connect(os.environ['POSTGRE_PASSWORD']) as conn:
    with conn.cursor() as cursor:
        sql='''
            SELECT 聯絡人id,客戶名稱,聯絡人姓名,電話,郵件
            FROM 聯絡人 LEFT JOIN 客戶 ON 聯絡人.客戶id = 客戶.客戶_id
        '''
        cursor.execute(sql)
        while True:
            oneItem = cursor.fetchone()
            if oneItem is not None:
                print(oneItem)
            else:
                break

(1, '遠傳電信', 'John Doe', '(408)-111-1234', 'john.doe@bluebird.dev')
(2, '遠傳電信', 'Jane Doe', '(408)-111-1235', 'jane.doe@bluebird.dev')
(3, '台灣大車隊', 'David Wright', '(408)-222-1234', 'david.wright@dolphin.dev')
